In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers.legacy import Adam
from PIL import Image
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout

In [2]:
# de cuales tenemos suficiente data?
#tenemos algunos...
#allaple-A -!
#yuner - a -!
#allaple.L
#instantaccess
#vB.at

In [3]:
# Directorio base donde se encuentran las imágenes de malware
base_dir = './images'

# Obtener la lista de familias de malware
familias = os.listdir(base_dir)

# Diccionario para almacenar las imágenes y etiquetas
data = {
    'imagenes': [],
    'etiquetas': []
}

familias


['dontovo_a',
 'instant_access',
 'vb_at',
 'dialplatform_b',
 'adialer_c',
 'agent_fyi',
 'allaple_a',
 'fakerean',
 'yuner_a',
 'lolyda_aa1',
 '.ipynb_checkpoints',
 'allaple_l',
 'auleron_gen_j']

In [4]:
# Recorrer cada familia de malware
for familia in familias:
    familia_dir = os.path.join(base_dir, familia)
    imagenes_familia = os.listdir(familia_dir)

In [5]:
# Tamaño común para redimensionar las imágenes
target_size = (224, 224)  # Ajusta el tamaño según tus necesidades

flag_counter = 0
# Recorrer cada familia de malware
for familia in familias:
    familia_dir = os.path.join(base_dir, familia)
    imagenes_familia = os.listdir(familia_dir)

    
    # Verificar si la familia tiene suficientes observaciones
    if len(imagenes_familia) >= 10:  # Ajusta el umbral según tus necesidades
        # Recorrer cada imagen de la familia

        counter_max_check_files_of_one_family = 0
        
        for imagen_nombre in imagenes_familia:

            ++counter_max_check_files_of_one_family
            if (counter_max_check_files_of_one_family < 100):        
                imagen_path = os.path.join(familia_dir, imagen_nombre)
                
                # Verificar si la ruta es un archivo antes de intentar abrirlo
                if os.path.isfile(imagen_path):
                    # Cargar la imagen y redimensionarla al tamaño común
                    imagen = Image.open(imagen_path)
                    imagen = imagen.resize(target_size)
                    imagen_array = np.array(imagen)
                    if(imagen_array.shape == (224,224)):
            
                    
                        # Agregar la imagen y la etiqueta al diccionario
                        data['imagenes'].append(imagen_array)
                        data['etiquetas'].append(flag_counter)
                    else:
                        print("esta raro")
        flag_counter = flag_counter + 1
# ...

In [6]:
counter_si = 0
counter_no = 0
for imagenes in data['imagenes']:
   
    if (imagenes.shape == (224,224)):
        counter_si += 1
    else:
        counter_no += 1

print(counter_si, counter_no)


5735 0


In [7]:
# Convertir las listas en arrays numpy
X = np.array(data['imagenes'])
y = np.array(data['etiquetas'])

In [71]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalizar los valores de píxeles entre 0 y 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


In [72]:
'''
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y_train)), activation='softmax')
])
'''

"\nmodel = Sequential([\n    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),\n    MaxPooling2D((2, 2)),\n    Conv2D(64, (3, 3), activation='relu'),\n    MaxPooling2D((2, 2)),\n    Conv2D(64, (3, 3), activation='relu'),\n    Flatten(),\n    Dense(64, activation='relu'),\n    Dense(len(np.unique(y_train)), activation='softmax')\n])\n"

In [73]:
# Agregar la dimensión de canal a las imágenes
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

print("Data type of X_train:", X_train.dtype)
print("Data type of X_test:", X_test.dtype)
# Construir el modelo de red neuronal
'''
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(224, 224, 1)),  # Ajusta la forma de entrada según tus imágenes
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Flatten(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(len(np.unique(y_train)), activation='softmax')
])
'''
my_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(224, 224, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.3),  # Increased dropout rate
    
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Dropout(0.3),  # Increased dropout rate
    
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),  # Added convolutional layer
    MaxPooling2D((2, 2)),
    Dropout(0.3),  # Increased dropout rate
    
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Dropout(0.3),  # Increased dropout rate
    
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),  # Increased units and added regularization
    Dropout(0.4),  # Increased dropout rate
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),  # Increased units and added regularization
    Dropout(0.4),  # Increased dropout rate
    Dense(len(np.unique(y_train)), activation='softmax')
])

'''
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(224, 224, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Flatten(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(len(np.unique(y_train)), activation='softmax')
])
'''

Data type of X_train: float32
Data type of X_test: float32


"\nmodel = Sequential([\n    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(224, 224, 1)),\n    MaxPooling2D((2, 2)),\n    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),\n    MaxPooling2D((2, 2)),\n    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),\n    Dropout(0.5),\n    Flatten(),\n    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),\n    Dropout(0.5),\n    Dense(len(np.unique(y_train)), activation='softmax')\n])\n"

In [74]:
# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
# Entrenar el modelo
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo en los datos de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

Epoch 1/5
126/126 [==============================] - 105s 834ms/step - loss: 4.2096 - accuracy: 0.4071 - val_loss: 1.8178 - val_accuracy: 0.6304
Epoch 2/5
126/126 [==============================] - 105s 836ms/step - loss: 1.4227 - accuracy: 0.7103 - val_loss: 0.9891 - val_accuracy: 0.9309
Epoch 3/5
126/126 [==============================] - 120s 954ms/step - loss: 0.8688 - accuracy: 0.9048 - val_loss: 0.6189 - val_accuracy: 0.9890
Epoch 4/5
126/126 [==============================] - 126s 1s/step - loss: 0.6511 - accuracy: 0.9492 - val_loss: 0.5531 - val_accuracy: 0.9849
Epoch 5/5
54/54 [==============================] - 14s 264ms/step - loss: 0.4180 - accuracy: 0.9913
Test Loss: 0.41796383261680603, Test Accuracy: 0.9912841320037842


In [75]:
my_y_test = y_test

In [ ]:
##parte 2 -------------------------

In [13]:
!pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.7 MB/s eta 0:00:0000:0100:010m


In [87]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier
from art.utils import load_dataset
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod
from keras.preprocessing.image import ImageDataGenerator
from art.attacks.evasion import FastGradientMethod
from art.attacks.poisoning import PoisoningAttackBackdoor
from art.estimators.classification import KerasClassifier
import numpy as np

In [34]:
# Cargar el conjunto de datos MNIST
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_dataset('mnist')

# Reshape de las imágenes de entrada
x_train = np.array([img.reshape((28, 28, 1)) for img in x_train])
x_test = np.array([img.reshape((28, 28, 1)) for img in x_test])


In [35]:
# Definir un modelo simple de CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])


In [36]:
# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [52]:
# Convertir etiquetas one-hot a escalares
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)

# Entrenar el modelo
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))


AxisError: axis 1 is out of bounds for array of dimension 1

In [39]:
# Crear el estimador de ART
estimator = KerasClassifier(model, clip_values=(min_pixel_value, max_pixel_value))


In [40]:
# Implementar el ataque FastGradientMethod
attack = FastGradientMethod(estimator=estimator, eps=0.2)
x_test_adv = attack.generate(x=x_test)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2024-05-05 18:03:33.596577: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_4/Softmax' id:28356 op device:{requested: '', assigned: ''} def:{{{node dense_4/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_4/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [44]:
# Evaluar el ataque
preds = np.argmax(estimator.predict(x_test), axis=1)
acc = np.sum(preds == np.argmax(y_test)) / y_test.shape[0]
print(f'Accuracy on original test examples: {acc * 100:.2f}%')

preds = np.argmax(estimator.predict(x_test_adv), axis=1)
acc = np.sum(preds == np.argmax(y_test)) / y_test.shape[0]
print(f'Accuracy on adversarial test examples: {acc * 100:.2f}%')

Accuracy on original test examples: 10.38%
Accuracy on adversarial test examples: 11.79%


In [47]:
from art.attacks.evasion import FastGradientMethod
from art.attacks.poisoning import PoisoningAttackBackdoor
from art.estimators.classification import KerasClassifier
import numpy as np


In [76]:
model = my_model

In [77]:
# Crear el estimador de ART con tu modelo
estimator = KerasClassifier(model, clip_values=(min_pixel_value, max_pixel_value))

In [78]:
#x_train = X_train.reshape(X_train.shape[0], 224, 224, 1)
X_test = X_test.reshape(X_test.shape[0], 224, 224, 1)

In [79]:
# Implementar el ataque de evasión (Fast Gradient Sign Method)
evasion_attack = FastGradientMethod(estimator=estimator, eps=0.2)
x_test_adv = evasion_attack.generate(x=X_test)


2024-05-05 18:34:04.934635: W tensorflow/c/c_api.cc:305] Operation '{name:'conv2d_11/kernel/Assign' id:29240 op device:{requested: '', assigned: ''} def:{{{node conv2d_11/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_11/kernel, conv2d_11/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [81]:
# Evaluar el ataque de evasión
preds = np.argmax(estimator.predict(X_test), axis=1)
acc = np.sum(preds == np.argmax(my_y_test)) / my_y_test.shape[0]
print(f'Accuracy on original test examples: {acc * 100:.2f}%')


Accuracy on original test examples: 0.00%


In [83]:
preds = np.argmax(estimator.predict(x_test_adv), axis=1)
acc = np.sum(preds == np.argmax(my_y_test)) / my_y_test.shape[0]
print(f'Accuracy on adversarial test examples (evasion attack): {acc * 100:.2f}%')


Accuracy on adversarial test examples (evasion attack): 0.00%


In [84]:
# Definir el patrón de la puerta trasera (backdoor)
backdoor_pattern = np.zeros((28, 28, 1))
backdoor_pattern[0:5, 0:5, :] = 1


In [95]:
from art.attacks.poisoning import PoisoningAttackBackdoor
from art.estimators.classification import KerasClassifier
from art.attacks.poisoning.perturbations import insert_pattern

# Crear el estimador de ART con tu modelo
estimator = KerasClassifier(model, clip_values=(min_pixel_value, max_pixel_value))

# Definir el patrón de la puerta trasera (backdoor)
backdoor_pattern = np.zeros((28, 28, 1))
backdoor_pattern[0:5, 0:5, :] = 1

# Definir la función de perturbación
perturbation = insert_pattern(backdoor_pattern, channels_first=False)

# Implementar el ataque de envenenamiento (Data Poisoning Attack)
poisoning_attack = PoisoningAttackBackdoor(perturbation)
x_train_poisoned, y_train_poisoned = poisoning_attack.poison(x_train, y_train)

# Entrenar tu modelo con los datos envenenados
model.fit(x_train_poisoned, y_train_poisoned, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# ... (resto del código)

ImportError: cannot import name 'insert_pattern' from 'art.attacks.poisoning.perturbations' (/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/art/attacks/poisoning/perturbations/__init__.py)